## Homework Assignment 2 - Evan Callaghan

## 1. Given 1000 records in a dataset, 1000 models are trained with 999 records as part of the training sample and the remaining 1 sample for testing, and the error rate is averaged out, this validation technique is called
#### C) This is an example of Leave-One-Out Cross Validation

## 2. In k-fold cross validation technique, the value of k being small could lead to which of the following in relation to the error rate
#### C) In k-fold cross validation technique, the value of k being small could lead to high bias and low variance

## 3. In k-fold cross validation technique, the value of k being large could lead to which of the following in relation to the error rate
#### B) In k-fold cross validation technique, the value of k being large could lead to low bias and high variance

## 4. Explain what regularization is and why it is useful.
#### Regularization is a method of regression that involves fitting a model using all input variables and the estimated coefficients being regularized/shrunken towards zero relative to their least squares estimates. It is useful because, effectively, regularization reduces the variance in predictions and protects against over-fitting in the model. 



In [9]:
## 5. a) Using the pandas library to read the csv data file and create a data-frame called heart

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [10]:
## 5. b)  Removing observations with missing values

heart = heart.dropna().reset_index()

In [15]:
## 5. c) Performing a 5-folds cross validation with the goal of measuring the performance, in terms of F1-score, of two competing models:

## Model 1) Using age, currentSmoker, totChol, sysBP, diaBP, BMI, heartRate, and glucose as the predictor variables, and TenYearCHD as the target variable to build a logistic regression
## model under the 5-folds cross validation framework

## Model 2) Using age, currentSmoker, totChol, BMI, heartRate, and glucose as the predictor variables, and TenYearCHD as the target variable to build a logistic regression model under 
## the 5-folds cross validation framework

X = heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

X = X.rename(columns = {0: 'age', 1: 'currentSmoker', 2: 'totChol', 3: 'sysBP', 4: 'diaBP', 5: 'BMI', 6: 'heartRate', 7: 'glucose'})

## Defining lists to store results 
md1_f1 = []
md2_f1 = []

kfold = KFold(n_splits = 5, shuffle = True)

for train_ix, test_ix in kfold.split(X):
    
    ## Defining the data for the first model
    X_train, X_val = X.loc[train_ix], X.loc[test_ix]
    Y_train, Y_val = Y.loc[train_ix], Y.loc[test_ix]
    
    ## Fitting the first logistic regression model
    logit_md1 = LogisticRegression().fit(X_train, Y_train)
    
    ## Predicting first model on validation set
    val_preds1 = logit_md1.predict_proba(X_val)[:, 1]
    val_preds1 = np.where(val_preds1 < 0.25, 0, 1)
    
    ## Computing the F1 score of the k-fold for Model 1
    md1_f1.append(f1_score(Y_val, val_preds1))
    
    
    ## Defining the data for the second model
    X_train_new = X_train.drop(columns = ['sysBP', 'diaBP'], axis = 1)
    X_val_new = X_val.drop(columns = ['sysBP', 'diaBP'], axis = 1)
    
    ## Fitting the second logistic regression model
    logit_md2 = LogisticRegression().fit(X_train_new, Y_train)
    
    ## Predicting second model on validation set
    val_preds2 = logit_md2.predict_proba(X_val_new)[:, 1]
    val_preds2 = np.where(val_preds2 < 0.25, 0, 1)
    
    ## Computing the F1 score of the k-fold for Model 2
    md2_f1.append(f1_score(Y_val, val_preds2))


In [16]:
## 5. d) Reporting the average F1-score of each of the models

print('Average F1 Score for Model 1:', np.mean(md1_f1))
print('Average F1 Score for Model 2:', np.mean(md2_f1))

## I would choose Model 1 to predict TenTearCHD because it has a higher F1-Score, indicating a more accurate model.

Average F1 Score for Model 1: 0.3379374450462012
Average F1 Score for Model 2: 0.3278212271928657
